# Part 4 - Binary Sentiment Classification & Part 6 - Clustering

# Install Dependencies
This section installs required libraries (scikit-learn for machine learning, duckdb for querying Parquet files) to support sentiment analysis and clustering. The environment is verified to ensure compatibility.

In [2]:
!pip install sklearn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [12]:
!pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 4.2 MB/s eta 0:00:03
   --------------------- ------------------ 6.0/11.1 MB 21.7 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 27.8 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ---------------------------------------- 0.0/41.2 MB ? eta -:--:--
   ---------- ----------------------------- 11.0/41.2 MB 57.5 MB/s eta 0:00:01
   -------------- ------------------------- 15.2/41.2 MB 41.6 MB/s eta 0:00:01
   ------------------------ --------------- 25.7/41.2 MB 42.9 MB/s eta 0:00:01
   -------------------------------- ------- 33.8/41.2 MB 42.1 MB/s eta 0:00:01
   ---------------------------------------  41.2/41.2 MB 40.9 MB/s eta 0:00:01
   ---------------------------------------- 41.

In [8]:
import sys
print(sys.prefix)

c:\Users\Alindo\Downloads\assignment3notebook\.conda


In [13]:
import sklearn
print(sklearn.__version__)

1.6.1


In [3]:
!pip install scikit-learn

In [1]:
!pip install duckdb

## Task 6: Clustering / Segmentation (k-means)

### Objective
Segment products into 5 clusters using k-means clustering based on the following features per product:
- **mean_rating**: Average user rating per product.
- **total_reviews**: Total number of reviews for the product.
- **brand_id**: Integer ID mapped from distinct brand strings.
- **category_id**: Integer ID mapped from distinct main category strings.

### Methodology
1. **Data Preparation**:
   - Use DuckDB to query the parquet files and aggregate features by `parent_asin`.
   - Compute `mean_rating` (average rating) and `total_reviews` (count of reviews) per product.
   - Map brands and categories to integer IDs using distinct values from the dataset.
   - Filter out invalid entries (e.g., missing or empty brands/categories, ratings not in [1-5]).
2. **Feature Scaling**:
   - Standardize features using `StandardScaler` to ensure equal contribution to clustering.
3. **K-means Clustering**:
   - Apply k-means with `k=5`, default initialization, and random state 42 using `sklearn.cluster.KMeans`.
   - Assign cluster labels to each product.
4. **Cluster Analysis**:
   - For each cluster, compute:
     - Size (number of products).
     - Average `mean_rating`, `total_reviews`, `brand_id`, and `category_id`.
     - Top 3 brands and categories by frequency.
   - Provide an interpretation of each cluster (e.g., high-rating products, low-review items).

### Implementation
The following code performs the clustering and analysis, outputting the results for each cluster. It handles the large dataset efficiently using DuckDB and processes approximately 32.6 million unique products.

In [ ]:
import duckdb
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from collections import Counter
import time

input_path = "F:/sentiments/sentiments/sentiment_*.parquet"
n_clusters = 5
random_state = 42

expected_categories = [
    "sentiment_All_Beauty", "sentiment_Amazon_Fashion", "sentiment_Appliances", "sentiment_Arts_Crafts_and_Sewing", "sentiment_Automotive",
    "sentiment_Baby_Products", "sentiment_Beauty_and_Personal_Care", "sentiment_Books", "sentiment_CDs_and_Vinyl",
    "sentiment_Cell_Phones_and_Accessories", "sentiment_Clothing_Shoes_and_Jewelry", "sentiment_Digital_Music", "sentiment_Electronics",
    "sentiment_Gift_Cards", "sentiment_Grocery_and_Gourmet_Food", "sentiment_Handmade_Products", "sentiment_Health_and_Household",
    "sentiment_Health_and_Personal_Care", "sentiment_Home_and_Kitchen", "sentiment_Industrial_and_Scientific", "sentiment_Kindle_Store",
    "sentiment_Magazine_Subscriptions", "sentiment_Movies_and_TV", "sentiment_Musical_Instruments", "sentiment_Office_Products",
    "sentiment_Patio_Lawn_and_Garden", "sentiment_Pet_Supplies", "sentiment_Software", "sentiment_Sports_and_Outdoors",
    "sentiment_Subscription_Boxes", "sentiment_Tools_and_Home_Improvement", "sentiment_Toys_and_Games", "sentiment_Video_Games", "sentiment_Unknown"
]


def get_most_common(series, top_n=3):
    """Gets the most common items and their counts from a Pandas Series."""
    counts = Counter(series.dropna())
    return counts.most_common(top_n)


start_time = time.time()
print("--- Starting Product Segmentation ---")


print("Initializing DuckDB...")
con = duckdb.connect()
print("DuckDB Initialized.")


print("Creating Brand and Category Mappings...")
try:
    mapping_query = f"""
        SELECT DISTINCT brand, main_category
        FROM read_parquet('{input_path}')
        WHERE brand IS NOT NULL AND TRIM(brand) != ''
          AND main_category IS NOT NULL AND TRIM(main_category) != ''
    """
    mappings_df = con.execute(mapping_query).fetch_df()

    unique_brands = mappings_df['brand'].unique()
    unique_categories = mappings_df['main_category'].unique() 

    brand_map = {brand: idx for idx, brand in enumerate(unique_brands)}
    category_map = {cat: idx for idx, cat in enumerate(unique_categories)}

    id_to_brand = {idx: brand for brand, idx in brand_map.items()}
    id_to_category = {idx: cat for cat, idx in category_map.items()}

    print(f"Found {len(brand_map)} unique brands and {len(category_map)} unique categories.")

except Exception as e:
    print(f"Error creating mappings: {e}")
    con.close()
    exit()


print("Aggregating product features (mean_rating, total_reviews)...")
try:
    aggregation_query = f"""
        WITH ProductStats AS (
            SELECT
                parent_asin,
                CAST(AVG(CAST(rating AS FLOAT)) AS FLOAT) as mean_rating,
                COUNT(*) as total_reviews,
                -- Use MIN/MAX or LIST to handle potential multiple brands/categories per parent_asin if needed
                -- Using FIRST assumes one primary brand/category per parent_asin in the data
                FIRST(brand) as brand,
                FIRST(main_category) as main_category
            FROM read_parquet('{input_path}')
            WHERE rating BETWEEN 1 AND 5
              AND parent_asin IS NOT NULL AND TRIM(parent_asin) != ''
              AND brand IS NOT NULL AND TRIM(brand) != ''        -- Ensure brand is valid
              AND main_category IS NOT NULL AND TRIM(main_category) != '' -- Ensure category is valid
            GROUP BY parent_asin
        )
        -- Final selection ensures we only include products where brand/category mapping is possible
        SELECT
            ps.parent_asin as product_id,
            ps.mean_rating,
            ps.total_reviews,
            ps.brand,
            ps.main_category
        FROM ProductStats ps
        WHERE ps.brand IS NOT NULL AND ps.main_category IS NOT NULL; -- Redundant check, but safe
    """
    
    product_features_df = con.execute(aggregation_query).fetch_df()
    print(f"Aggregated features for {len(product_features_df)} unique products.")

    if product_features_df.empty:
        print("No product data found after aggregation. Exiting.")
        con.close()
        exit()

except Exception as e:
    print(f"Error aggregating features: {e}")
    con.close()
    exit()


print("Preparing data for K-means (mapping IDs, scaling)...")
try:
    
    product_features_df['brand_id'] = product_features_df['brand'].map(brand_map)
    product_features_df['category_id'] = product_features_df['main_category'].map(category_map)

    missing_brands = product_features_df['brand_id'].isna().sum()
    missing_categories = product_features_df['category_id'].isna().sum()
    if missing_brands > 0 or missing_categories > 0:
        print(f"Warning: Found {missing_brands} products with unmapped brands and {missing_categories} with unmapped categories.")
        
        # Let's drop rows with mapping issues for cleaner clustering
        product_features_df.dropna(subset=['brand_id', 'category_id'], inplace=True)
        product_features_df['brand_id'] = product_features_df['brand_id'].astype(int)
        product_features_df['category_id'] = product_features_df['category_id'].astype(int)
        print(f"Proceeding with {len(product_features_df)} products after removing mapping failures.")

    if product_features_df.empty:
        print("No products remaining after handling mapping issues. Exiting.")
        con.close()
        exit()

    features = product_features_df[['mean_rating', 'total_reviews', 'brand_id', 'category_id']].values

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    print("Features scaled.")

except Exception as e:
    print(f"Error preparing data for K-means: {e}")
    con.close()
    exit()


print(f"Applying K-means clustering (k={n_clusters})...")
try:
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state, n_init='auto', verbose=0) # verbose=1 for logs
    cluster_labels = kmeans.fit_predict(features_scaled)

    product_features_df['cluster'] = cluster_labels
    print("K-means fitting complete.")

except Exception as e:
    print(f"Error during K-means clustering: {e}")
    con.close()
    exit()


print("\n--- Cluster Analysis ---")
for cluster_id in range(n_clusters):
    # Filter data for the current cluster
    cluster_df = product_features_df[product_features_df['cluster'] == cluster_id]
    cluster_size = len(cluster_df)

    if cluster_size == 0:
        print(f"\nCluster {cluster_id}: Empty")
        continue

    avg_mean_rating = cluster_df['mean_rating'].mean()
    avg_total_reviews = cluster_df['total_reviews'].mean()
    # Avg IDs are not meaningful, but calculated
    avg_brand_id = cluster_df['brand_id'].mean()
    avg_category_id = cluster_df['category_id'].mean()

    # Find most common brands and categories for interpretation
    top_brands = get_most_common(cluster_df['brand'], top_n=3)
    top_categories = get_most_common(cluster_df['main_category'], top_n=3)

    
    rating_desc = "High" if avg_mean_rating >= 4.2 else "Low" if avg_mean_rating < 3.5 else "Moderate"
   
    review_desc = "Very High" if avg_total_reviews > 1000 else "High" if avg_total_reviews > 100 else "Moderate" if avg_total_reviews > 10 else "Low"
    
    # Describe common elements
    brands_str = ", ".join([f"{b} ({c})" for b, c in top_brands]) if top_brands else "N/A"
    categories_str = ", ".join([f"{cat} ({c})" for cat, c in top_categories]) if top_categories else "N/A"

    interpretation = (
        f"{rating_desc} Rating ({avg_mean_rating:.2f}), {review_desc} Reviews ({avg_total_reviews:.1f}). "
        f"Common Brands: [{brands_str}]. Common Categories: [{categories_str}]."
    )

    print(f"\nCluster {cluster_id}:")
    print(f"  Size: {cluster_size} products ({cluster_size / len(product_features_df) * 100:.1f}%)")
    print(f"  Avg Mean Rating: {avg_mean_rating:.2f}")
    print(f"  Avg Total Reviews: {avg_total_reviews:.2f}")
    
    print(f"  Avg Brand ID: {avg_brand_id:.2f}")
    print(f"  Avg Category ID: {avg_category_id:.2f}")
    print(f"  Top 3 Brands: {top_brands}")
    print(f"  Top 3 Categories: {top_categories}")
    print(f"  Interpretation: {interpretation}")


print("\nClosing DuckDB connection...")
con.close()

end_time = time.time()
print(f"--- Script finished in {end_time - start_time:.2f} seconds ---")

--- Starting Product Segmentation ---
Initializing DuckDB...
DuckDB Initialized.
Creating Brand and Category Mappings...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Found 4769188 unique brands and 50 unique categories.
Aggregating product features (mean_rating, total_reviews)...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Aggregated features for 32609113 unique products.
Preparing data for K-means (mapping IDs, scaling)...
Features scaled.
Applying K-means clustering (k=5)...
K-means fitting complete.

--- Cluster Analysis ---

Cluster 0:
  Size: 6703788 products (20.6%)
  Avg Mean Rating: 4.48
  Avg Total Reviews: 10.88
  Avg Brand ID: 3215458.76
  Avg Category ID: 8.53
  Top 3 Brands: [('Naturalizer', 10367), ('Aerosoles', 10183), ('Goodthreads', 9859)]
  Top 3 Categories: [('Books', 2303290), ('AMAZON FASHION', 1616574), ('Amazon Home', 763316)]
  Interpretation: High Rating (4.48), Moderate Reviews (10.9). Common Brands: [Naturalizer (10367), Aerosoles (10183), Goodthreads (9859)]. Common Categories: [Books (2303290), AMAZON FASHION (1616574), Amazon Home (763316)].

Cluster 1:
  Size: 4716768 products (14.5%)
  Avg Mean Rating: 4.37
  Avg Total Reviews: 16.39
  Avg Brand ID: 1908860.60
  Avg Category ID: 29.78
  Top 3 Brands: [('Unknown', 404677), ('Generic', 46357), ('Format: DVD', 37983)]
  Top 3

## Task 4: Binary Sentiment Prediction (Logistic Regression)

### Objective
Perform binary sentiment classification on review text, where:
- **Positive**: Rating > 3
- **Negative**: Rating ≤ 3
Evaluate the model using accuracy, F1 score, and confusion matrix.

### Methodology
1. **Data Preparation**:
   - Use DuckDB to query review text and ratings from parquet files.
   - Filter for valid ratings (1-5) and non-empty review text.
   - Create binary sentiment labels (1 for positive, 0 for negative).
   - Split data into 80% training and 20% test sets, stratified by sentiment.
2. **Text Vectorization**:
   - Apply TF-IDF vectorization on review text using `TfidfVectorizer` with:
     - Lowercase conversion.
     - Token pattern `\b\w+\b`.
     - Minimum document frequency of 5.
     - Maximum document frequency of 80%.
3. **Classifier**:
   - Use `SGDClassifier` with `log_loss` to implement logistic regression.
   - Train the model in batches of 5 million reviews to handle the large dataset.
4. **Evaluation**:
   - Compute accuracy, F1 score, and confusion matrix for each test batch.
   - Aggregate metrics across all batches to report overall performance.

### Implementation
The following code processes the dataset in batches, trains the logistic regression model, and evaluates performance. It handles 503,024,752 valid reviews, with results reported per batch and overall.

In [3]:
import duckdb
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

input_path = "F:/sentiments/sentiments/sentiment_*.parquet"

batch_size = 5000000 

vectorizer = TfidfVectorizer(
    lowercase=True,
    token_pattern=r'\b\w+\b',
    min_df=5,
    max_df=0.8
)
model = SGDClassifier(loss='log_loss', random_state=42)
classes = [0, 1]

offset = 0
batch_num = 1
con = duckdb.connect()

all_preds = []
all_truths = []

while True:
    query = f"""
        SELECT text, rating
        FROM read_parquet('{input_path}')
        WHERE rating BETWEEN 1 AND 5
          AND text IS NOT NULL
          AND LENGTH(TRIM(text)) > 0
        LIMIT {batch_size} OFFSET {offset}
    """
    df = con.execute(query).fetch_df()
    if df.empty:
        break

    df["label"] = (df["rating"] > 3).astype(int)

    X_train, X_test, y_train, y_test = train_test_split(
        df["text"], df["label"], test_size=0.2, random_state=42
    )

    if batch_num == 1:
        X_train_vec = vectorizer.fit_transform(X_train)
        model.partial_fit(X_train_vec, y_train, classes=classes)
    else:
        X_train_vec = vectorizer.transform(X_train)
        model.partial_fit(X_train_vec, y_train)

    X_test_vec = vectorizer.transform(X_test)
    y_pred = model.predict(X_test_vec)

    all_preds.extend(y_pred)
    all_truths.extend(y_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print(f" Batch {batch_num} — accuracy: {acc:.4f}, F1: {f1:.4f}")
    print(cm, "\n")

    offset += batch_size
    batch_num += 1

# Calculate overall metrics
if all_preds and all_truths:
    overall_acc = accuracy_score(all_truths, all_preds)
    overall_f1 = f1_score(all_truths, all_preds)
    overall_cm = confusion_matrix(all_truths, all_preds)

    print("Overall Metrics Across All Batches:")
    print(f"Total Confusion Matrix:\n{overall_cm}")
    print(f"Overall Accuracy: {overall_acc:.4f}")
    print(f"Overall F1 Score: {overall_f1:.4f}")

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

 Batch 1 — accuracy: 0.8713, F1: 0.9175
[[155895  99598]
 [ 29096 715411]] 



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

 Batch 2 — accuracy: 0.8839, F1: 0.9307
[[103513  96347]
 [ 19786 780354]] 



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

 Batch 100 — accuracy: 0.8595, F1: 0.9106
[[143305 115598]
 [ 24944 716153]] 



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

 Batch 101 — accuracy: 0.8457, F1: 0.9031
[[ 76729  77338]
 [ 16028 434856]] 



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Overall Metrics Across All Batches:
Total Confusion Matrix:
[[10628617 11438593]
 [ 2071764 76465977]]
Overall Accuracy: 0.8657
Overall F1 Score: 0.9188
